In [3]:
%%capture
!pip install evaluate
!pip install transformers
!pip install datasets

**CBi-GRU**

In [35]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. CBi-GRU Model
# # =========================================================
# class CBiGRUModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(CBiGRUModel, self).__init__()

#         # Convolutional Layer for feature extraction
#         self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)

#         # Embedding Layer
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         # GRU Layer (Bidirectional)
#         self.gru = nn.GRU(128, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

#         # Fully Connected Layer
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         # Embedding the input sequence
#         embeds = self.embedding(input_ids)   # (batch, seq_len, embed_dim)
#         embeds = embeds.permute(0, 2, 1)  # (batch, embed_dim, seq_len)

#         # Convolutional layers for feature extraction
#         x = self.conv1(embeds)
#         x = self.conv2(x)
#         x = x.permute(0, 2, 1)  # (batch, seq_len, 128)

#         # GRU for sequential learning
#         gru_out, _ = self.gru(x)   # (batch, seq_len, hidden_dim*2)

#         # Linear layer for classification
#         logits = self.classifier(gru_out)  # (batch, seq_len, num_labels)

#         # Compute loss if labels are provided
#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = CBiGRUModel(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 20
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-2638687586.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/20 | Train Loss: 0.9132 | Val Loss: 0.8014
Epoch 2/20 | Train Loss: 0.7668 | Val Loss: 0.7516
Epoch 3/20 | Train Loss: 0.7250 | Val Loss: 0.7236
Epoch 4/20 | Train Loss: 0.7039 | Val Loss: 0.7152
Epoch 5/20 | Train Loss: 0.6914 | Val Loss: 0.7131
Epoch 6/20 | Train Loss: 0.6824 | Val Loss: 0.6995
Epoch 7/20 | Train Loss: 0.6754 | Val Loss: 0.6923
Epoch 8/20 | Train Loss: 0.6692 | Val Loss: 0.6927
Epoch 9/20 | Train Loss: 0.6643 | Val Loss: 0.6894
Epoch 10/20 | Train Loss: 0.6602 | Val Loss: 0.6875
Epoch 11/20 | Train Loss: 0.6558 | Val Loss: 0.6838
Epoch 12/20 | Train Loss: 0.6532 | Val Loss: 0.6779
Epoch 13/20 | Train Loss: 0.6513 | Val Loss: 0.6729
Epoch 14/20 | Train Loss: 0.6480 | Val Loss: 0.6748
Epoch 15/20 | Train Loss: 0.6470 | Val Loss: 0.6756
Epoch 16/20 | Train Loss: 0.6456 | Val Loss: 0.6754
Epoch 17/20 | Train Loss: 0.6464 | Val Loss: 0.6796
Epoch 18/20 | Train Loss: 0.6465 | Val Loss: 0.6755
Epoch 19/20 | Train L

In [36]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7181    0.7669    0.7417    646157
           E     0.8219    0.8608    0.8409    563979
           H     0.8498    0.9226    0.8847    682331
           B     0.7744    0.2266    0.3506     26516
           G     0.6756    0.4581    0.5460     72668
           I     0.6214    0.1680    0.2645       381
           T     0.6414    0.5681    0.6025    255238
           S     0.5907    0.4300    0.4977    210849

    accuracy                         0.7671   2458119
   macro avg     0.7117    0.5501    0.5911   2458119
weighted avg     0.7589    0.7671    0.7587   2458119

Q8 Accuracy: 0.7671


**DNSS2**

In [32]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. DNSS2 Model (Modified with CNN and BiLSTM)
# # =========================================================
# class DNSS2Model(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(DNSS2Model, self).__init__()

#         # Embedding layer
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         # CNN Layer
#         self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)  # Conv1d for sequence data
#         self.pool = nn.MaxPool1d(2)

#         # BiLSTM Layer
#         self.lstm = nn.LSTM(128, hidden_dim, num_layers=2, bidirectional=True, batch_first=True)

#         # Output layer (classifier)
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)   # (batch, seq_len, embed_dim)
#         embeds = embeds.transpose(1, 2)      # (batch, embed_dim, seq_len)

#         conv_out = self.conv1(embeds)        # (batch, 128, seq_len)
#         pooled_out = self.pool(conv_out)     # (batch, 128, seq_len//2)

#         lstm_out, _ = self.lstm(pooled_out.transpose(1, 2))  # (batch, seq_len//2, hidden*2)

#         logits = self.classifier(lstm_out)   # (batch, seq_len//2, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             # Adjust labels to match the sequence length after pooling
#             labels = labels[:, :logits.size(1)].reshape(-1)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels)

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = DNSS2Model(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-1321830254.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/10 | Train Loss: 1.0282 | Val Loss: 0.8467
Epoch 2/10 | Train Loss: 0.7714 | Val Loss: 0.7419
Epoch 3/10 | Train Loss: 0.6893 | Val Loss: 0.6963
Epoch 4/10 | Train Loss: 0.6406 | Val Loss: 0.6670
Epoch 5/10 | Train Loss: 0.6065 | Val Loss: 0.6452
Epoch 6/10 | Train Loss: 0.5797 | Val Loss: 0.6284
Epoch 7/10 | Train Loss: 0.5588 | Val Loss: 0.6149
Epoch 8/10 | Train Loss: 0.5406 | Val Loss: 0.6050
Epoch 9/10 | Train Loss: 0.5242 | Val Loss: 0.5957
Epoch 10/10 | Train Loss: 0.5099 | Val Loss: 0.5881


In [33]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7800    0.8156    0.7974    394168
           E     0.8386    0.8847    0.8611    294137
           H     0.8663    0.9311    0.8976    357253
           B     0.7664    0.3570    0.4871     14114
           G     0.7074    0.5496    0.6186     35551
           I     0.5123    0.2902    0.3705       286
           T     0.6848    0.6021    0.6408    137704
           S     0.6609    0.4922    0.5642    112256

    accuracy                         0.8006   1345469
   macro avg     0.7271    0.6153    0.6547   1345469
weighted avg     0.7939    0.8006    0.7943   1345469

Q8 Accuracy: 0.8006


**F1DCNN-SS**

In [26]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. F1DCNN-SS Model
# # =========================================================
# class F1DCNNSSModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_filters, filter_sizes, num_labels, pad_idx=0):
#         super(F1DCNNSSModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.filter_sizes = filter_sizes
#         self.num_filters = num_filters

#         # CNN layers for feature extraction
#         self.convs = nn.ModuleList([
#             nn.Conv1d(embed_dim, num_filters, kernel_size=f, padding=(f - 1) // 2) for f in filter_sizes
#         ])

#         # Fully connected classifier
#         self.fc = nn.Linear(num_filters * len(filter_sizes), num_labels)

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)   # (batch, seq_len, embed_dim)
#         embeds = embeds.transpose(1, 2)      # (batch, embed_dim, seq_len)

#         # Apply convolutional layers with padding to maintain sequence length
#         conv_outs = [torch.relu(conv(embeds)) for conv in self.convs]  # List of (batch, num_filters, seq_len)

#         # Ensure all conv_outs have the same sequence length before concatenation
#         # This might be necessary if padding doesn't perfectly align due to different kernel sizes
#         min_seq_len = min([out.size(2) for out in conv_outs])
#         conv_outs_padded = [out[:, :, :min_seq_len] for out in conv_outs]


#         # Concatenate along the channel dimension
#         cnn_out = torch.cat(conv_outs_padded, dim=1) # (batch, num_filters * len(filter_sizes), min_seq_len)

#         # Transpose for linear layer
#         cnn_out = cnn_out.transpose(1, 2) # (batch, min_seq_len, num_filters * len(filter_sizes))

#         # Apply linear layer at each position
#         logits = self.fc(cnn_out) # (batch, min_seq_len, num_labels)

#         # Loss calculation for sequence labeling
#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             # Reshape logits and labels to (total_elements, ...)
#             # Need to also trim labels to min_seq_len
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels[:, :min_seq_len].contiguous().view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# num_filters = 128
# filter_sizes = [3, 4, 5]
# num_labels = len(label_map)

# model = F1DCNNSSModel(vocab_size, embed_dim, num_filters, filter_sizes, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-612674803.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 1.2864 | Val Loss: 1.2620
Epoch 2/5 | Train Loss: 1.2560 | Val Loss: 1.2494
Epoch 3/5 | Train Loss: 1.2489 | Val Loss: 1.2477
Epoch 4/5 | Train Loss: 1.2453 | Val Loss: 1.2422
Epoch 5/5 | Train Loss: 1.2425 | Val Loss: 1.2366


In [27]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           C     0.5393    0.5938    0.5652    642640
           E     0.5664    0.6257    0.5946    560805
           H     0.5568    0.6960    0.6187    678112
           B     0.8044    0.0628    0.1165     26378
           G     0.6012    0.0986    0.1695     72270
           I     0.0000    0.0000    0.0000       380
           T     0.4455    0.3467    0.3900    253734
           S     0.4787    0.1321    0.2071    209560

    accuracy                         0.5438   2443879
   macro avg     0.4990    0.3195    0.3327   2443879
weighted avg     0.5401    0.5438    0.5213   2443879

Q8 Accuracy: 0.5438


**DeepACLSTM**

In [21]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}


# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. DeepACLSTM Model
# # =========================================================
# class DeepACLSTM(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(DeepACLSTM, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.conv1 = nn.Conv1d(embed_dim, 64, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
#         self.lstm = nn.LSTM(128, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
#         self.attention = nn.Linear(hidden_dim * 2, 1)  # attention layer
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)   # (batch, seq_len, embed_dim)
#         x = embeds.permute(0, 2, 1)  # Change shape for convolution (batch, embed_dim, seq_len)
#         x = self.conv1(x)           # (batch, 64, seq_len)
#         x = self.conv2(x)           # (batch, 128, seq_len)
#         x = x.permute(0, 2, 1)      # (batch, seq_len, 128)
#         lstm_out, _ = self.lstm(x)  # (batch, seq_len, hidden*2)

#         # Attention mechanism
#         attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
#         context = torch.sum(attn_weights * lstm_out, dim=1)

#         logits = self.classifier(lstm_out)    # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = DeepACLSTM(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-1045192263.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/10 | Train Loss: 0.9027 | Val Loss: 0.7671
Epoch 2/10 | Train Loss: 0.7226 | Val Loss: 0.7034
Epoch 3/10 | Train Loss: 0.6694 | Val Loss: 0.6721
Epoch 4/10 | Train Loss: 0.6395 | Val Loss: 0.6488
Epoch 5/10 | Train Loss: 0.6196 | Val Loss: 0.6386
Epoch 6/10 | Train Loss: 0.6047 | Val Loss: 0.6320
Epoch 7/10 | Train Loss: 0.5939 | Val Loss: 0.6246
Epoch 8/10 | Train Loss: 0.5849 | Val Loss: 0.6153
Epoch 9/10 | Train Loss: 0.5766 | Val Loss: 0.6083
Epoch 10/10 | Train Loss: 0.5695 | Val Loss: 0.6037


In [22]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7554    0.7990    0.7766    646157
           E     0.8582    0.8786    0.8683    563979
           H     0.8706    0.9343    0.9013    682331
           B     0.7798    0.3044    0.4379     26516
           G     0.7159    0.5293    0.6086     72668
           I     0.4459    0.3675    0.4029       381
           T     0.6723    0.6290    0.6499    255238
           S     0.6509    0.4995    0.5653    210849

    accuracy                         0.7981   2458119
   macro avg     0.7186    0.6177    0.6513   2458119
weighted avg     0.7924    0.7981    0.7923   2458119

Q8 Accuracy: 0.7981


**CNNH PSS**

In [18]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data_cleaned['seq']   # amino acid sequences
# y = pss_data_cleaned['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. CNN Model for PSSP (Adapted for Sequence Labeling)
# # =========================================================
# class CNNModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_filters, kernel_size, num_labels, pad_idx=0):
#         super(CNNModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         # Adjusted CNN layers for sequence labeling
#         # Use padding to maintain sequence length
#         padding = (kernel_size - 1) // 2
#         self.conv1 = nn.Conv1d(embed_dim, num_filters, kernel_size=kernel_size, padding=padding)
#         # Remove pooling and flattening
#         self.classifier = nn.Linear(num_filters, num_labels) # Classifier for each position

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)  # (batch, seq_len, embed_dim)
#         embeds = embeds.permute(0, 2, 1)   # Change shape to (batch, embed_dim, seq_len)

#         x = self.conv1(embeds)       # (batch, num_filters, seq_len) after padding

#         x = x.transpose(1, 2) # (batch, seq_len, num_filters)
#         logits = self.classifier(x)  # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             # Reshape logits and labels for CrossEntropyLoss
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# num_filters = 64
# filter_size = 3 # Using a kernel size of 3
# num_labels = len(label_map)

# model = CNNModel(vocab_size, embed_dim, num_filters, filter_size, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-3066850092.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 1.4681 | Val Loss: 1.4626
Epoch 2/5 | Train Loss: 1.4639 | Val Loss: 1.4649
Epoch 3/5 | Train Loss: 1.4633 | Val Loss: 1.4612
Epoch 4/5 | Train Loss: 1.4629 | Val Loss: 1.4617
Epoch 5/5 | Train Loss: 1.4628 | Val Loss: 1.4609


In [19]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           C     0.4085    0.5338    0.4628    646157
           E     0.4545    0.4384    0.4463    563979
           H     0.4315    0.5943    0.5000    682331
           B     0.0000    0.0000    0.0000     26516
           G     0.0000    0.0000    0.0000     72668
           I     0.0000    0.0000    0.0000       381
           T     0.3403    0.1733    0.2297    255238
           S     0.0000    0.0000    0.0000    210849

    accuracy                         0.4239   2458119
   macro avg     0.2044    0.2175    0.2049   2458119
weighted avg     0.3668    0.4239    0.3867   2458119

Q8 Accuracy: 0.4239


**eCRRNN**

In [14]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. eCRRNN Model (Modified Architecture)
# # =========================================================
# class eCRRNNModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(eCRRNNModel, self).__init__()

#         # Embedding layer
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         # Convolutional layer (CNN part of eCRRNN)
#         self.conv1 = nn.Conv1d(embed_dim, 256, kernel_size=5, padding=2)  # Add 1D convolution
#         self.conv2 = nn.Conv1d(256, 512, kernel_size=5, padding=2)

#         # Recurrent layer (GRU or LSTM, RNN part of eCRRNN)
#         self.gru = nn.GRU(512, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

#         # Fully connected layer for classification
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         # Embedding layer
#         embeds = self.embedding(input_ids)   # (batch, seq_len, embed_dim)

#         # Reshape for Conv1D input: (batch, embed_dim, seq_len)
#         embeds = embeds.permute(0, 2, 1)

#         # Convolutional layers
#         conv_out = self.conv1(embeds)         # (batch, 256, seq_len)
#         conv_out = torch.relu(conv_out)
#         conv_out = self.conv2(conv_out)       # (batch, 512, seq_len)
#         conv_out = torch.relu(conv_out)

#         # Reshape for GRU input: (batch, seq_len, 512)
#         conv_out = conv_out.permute(0, 2, 1)

#         # Recurrent layer (GRU)
#         gru_out, _ = self.gru(conv_out)      # (batch, seq_len, hidden*2)

#         # Classification layer
#         logits = self.classifier(gru_out)    # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = eCRRNNModel(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-908720724.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/10 | Train Loss: 0.7933 | Val Loss: 0.6482
Epoch 2/10 | Train Loss: 0.6041 | Val Loss: 0.5976
Epoch 3/10 | Train Loss: 0.5605 | Val Loss: 0.5751
Epoch 4/10 | Train Loss: 0.5369 | Val Loss: 0.5618
Epoch 5/10 | Train Loss: 0.5185 | Val Loss: 0.5512
Epoch 6/10 | Train Loss: 0.5027 | Val Loss: 0.5394
Epoch 7/10 | Train Loss: 0.4873 | Val Loss: 0.5252
Epoch 8/10 | Train Loss: 0.4721 | Val Loss: 0.5112
Epoch 9/10 | Train Loss: 0.4580 | Val Loss: 0.5020
Epoch 10/10 | Train Loss: 0.4453 | Val Loss: 0.4932


In [15]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7946    0.8430    0.8181    646157
           E     0.8905    0.9076    0.8990    563979
           H     0.9067    0.9407    0.9234    682331
           B     0.7577    0.4420    0.5583     26516
           G     0.7363    0.6469    0.6887     72668
           I     0.7674    0.1732    0.2827       381
           T     0.7248    0.6986    0.7115    255238
           S     0.7201    0.5615    0.6310    210849

    accuracy                         0.8356   2458119
   macro avg     0.7873    0.6517    0.6891   2458119
weighted avg     0.8319    0.8356    0.8320   2458119

Q8 Accuracy: 0.8356


**CRRNN**

In [11]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. CRNN Model (Convolutional Recurrent Neural Network)
# # =========================================================
# class CRNNModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(CRNNModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         # Convolutional layer
#         self.conv1 = nn.Conv1d(embed_dim, 64, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)

#         # GRU layer
#         self.gru = nn.GRU(128, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)

#         # Fully connected layer
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)  # (batch, seq_len, embed_dim)
#         embeds = embeds.transpose(1, 2)  # (batch, embed_dim, seq_len) for Conv1d

#         # Apply convolution
#         conv_out = torch.relu(self.conv1(embeds))
#         conv_out = torch.relu(self.conv2(conv_out))

#         # Recurrent layer (GRU)
#         gru_out, _ = self.gru(conv_out.transpose(1, 2))  # (batch, seq_len, hidden*2)

#         logits = self.classifier(gru_out)  # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = CRNNModel(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipython-input-4072971866.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/content/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/10 | Train Loss: 0.9124 | Val Loss: 0.7644
Epoch 2/10 | Train Loss: 0.7190 | Val Loss: 0.7148
Epoch 3/10 | Train Loss: 0.6705 | Val Loss: 0.6754
Epoch 4/10 | Train Loss: 0.6468 | Val Loss: 0.6687
Epoch 5/10 | Train Loss: 0.6318 | Val Loss: 0.6478
Epoch 6/10 | Train Loss: 0.6204 | Val Loss: 0.6497
Epoch 7/10 | Train Loss: 0.6114 | Val Loss: 0.6389
Epoch 8/10 | Train Loss: 0.6028 | Val Loss: 0.6322
Epoch 9/10 | Train Loss: 0.5953 | Val Loss: 0.6258
Epoch 10/10 | Train Loss: 0.5882 | Val Loss: 0.6193


In [12]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7612    0.7751    0.7681    646157
           E     0.8399    0.8857    0.8622    563979
           H     0.8621    0.9359    0.8975    682331
           B     0.7829    0.2871    0.4201     26516
           G     0.6621    0.5495    0.6005     72668
           I     0.7204    0.1759    0.2827       381
           T     0.6565    0.6195    0.6375    255238
           S     0.6430    0.4689    0.5423    210849

    accuracy                         0.7907   2458119
   macro avg     0.7410    0.5872    0.6264   2458119
weighted avg     0.7836    0.7907    0.7839   2458119

Q8 Accuracy: 0.7907


**BiGRU Model**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}


# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 4. Encoding Sequences & Labels
# # =========================================================
# # Amino acid vocabulary
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 5. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 6. BiGRU Model
# # =========================================================
# class BiGRUModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(BiGRUModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=1,
#                           bidirectional=True, batch_first=True)
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)   # (batch, seq_len, embed_dim)
#         gru_out, _ = self.gru(embeds)        # (batch, seq_len, hidden*2)
#         logits = self.classifier(gru_out)    # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 7. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = BiGRUModel(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipykernel_19/267209145.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 0.9290 | Val Loss: 0.8078
Epoch 2/5 | Train Loss: 0.7690 | Val Loss: 0.7564
Epoch 3/5 | Train Loss: 0.7313 | Val Loss: 0.7406
Epoch 4/5 | Train Loss: 0.7181 | Val Loss: 0.7356
Epoch 5/5 | Train Loss: 0.7116 | Val Loss: 0.7303


In [ ]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten but ignore padding (-100 in labels)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7121    0.7448    0.7281    646157
           E     0.7918    0.8615    0.8252    563979
           H     0.8415    0.9027    0.8710    682331
           B     0.7604    0.2170    0.3377     26516
           G     0.6293    0.4475    0.5230     72668
           I     0.6638    0.2021    0.3099       381
           T     0.6006    0.5581    0.5786    255238
           S     0.5834    0.3875    0.4657    210849

    accuracy                         0.7508   2458119
   macro avg     0.6978    0.5402    0.5799   2458119
weighted avg     0.7417    0.7508    0.7417   2458119

Q8 Accuracy: 0.7508


**CNN Model**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}


# # =========================================================
# # 3. Train / Val / Test Split
# # =========================================================
# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 3. Encoding Sequences & Labels
# # =========================================================
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 4. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 5. CNN Model
# # =========================================================
# class CNNModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_labels, pad_idx=0):
#         super(CNNModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         # 1D Convolution layers
#         self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv1d(128, 128, kernel_size=3, padding=1)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(128, num_labels)

#     def forward(self, input_ids, labels=None):
#         # input_ids: (batch, seq_len)
#         x = self.embedding(input_ids)       # (batch, seq_len, embed_dim)
#         x = x.transpose(1, 2)              # (batch, embed_dim, seq_len) for Conv1d
#         x = self.relu(self.conv1(x))        # (batch, 128, seq_len)
#         x = self.relu(self.conv2(x))        # (batch, 128, seq_len)
#         x = x.transpose(1, 2)              # back to (batch, seq_len, 128)
#         x = self.dropout(x)
#         logits = self.classifier(x)        # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 6. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# num_labels = len(label_map)

# model = CNNModel(vocab_size, embed_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipykernel_19/1134077729.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 1.3158 | Val Loss: 1.2569
Epoch 2/5 | Train Loss: 1.2729 | Val Loss: 1.2381
Epoch 3/5 | Train Loss: 1.2617 | Val Loss: 1.2220
Epoch 4/5 | Train Loss: 1.2559 | Val Loss: 1.2185
Epoch 5/5 | Train Loss: 1.2519 | Val Loss: 1.2133


In [ ]:
# from sklearn.metrics import classification_report, accuracy_score

# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids, labels=None)  # ✅ just logits
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten and remove padding
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# # Classification report
# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# # Q8 accuracy
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           C     0.5248    0.6276    0.5716    646157
           E     0.5922    0.6111    0.6015    563979
           H     0.5529    0.7333    0.6305    682331
           B     0.8668    0.0672    0.1248     26516
           G     0.7481    0.1018    0.1792     72668
           I     0.0000    0.0000    0.0000       381
           T     0.5114    0.3026    0.3802    255238
           S     0.6371    0.1071    0.1834    210849

    accuracy                         0.5531   2458119
   macro avg     0.5542    0.3188    0.3339   2458119
weighted avg     0.5665    0.5531    0.5251   2458119

Q8 Accuracy: 0.5531


**BiLSTM Model**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data_cleaned['seq']
# y = pss_data_cleaned['sst8']

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 3. Encoding Sequences & Labels
# # =========================================================
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1  # unknown amino acid
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128

# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 4. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 5. BiLSTM Model
# # =========================================================
# class BiLSTMModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=0):
#         super(BiLSTMModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=1,
#                             bidirectional=True, batch_first=True)
#         self.classifier = nn.Linear(hidden_dim * 2, num_labels)

#     def forward(self, input_ids, labels=None):
#         embeds = self.embedding(input_ids)        # (batch, seq_len, embed_dim)
#         lstm_out, _ = self.lstm(embeds)          # (batch, seq_len, hidden*2)
#         logits = self.classifier(lstm_out)       # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 6. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# hidden_dim = 256
# num_labels = len(label_map)

# model = BiLSTMModel(vocab_size, embed_dim, hidden_dim, num_labels, pad_idx=pad_idx).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epochs = 5
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)

#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

# # =========================================================
# # 7. Evaluation: Q8 Accuracy & Classification Report
# # =========================================================
# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids)  # ✅ BiLSTM returns only logits here
#         preds = torch.argmax(logits, dim=-1)

#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))

# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

/tmp/ipykernel_19/244902210.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 0.9270 | Val Loss: 0.7939
Epoch 2/5 | Train Loss: 0.7450 | Val Loss: 0.7245
Epoch 3/5 | Train Loss: 0.6902 | Val Loss: 0.6953
Epoch 4/5 | Train Loss: 0.6622 | Val Loss: 0.6790
Epoch 5/5 | Train Loss: 0.6463 | Val Loss: 0.6665
              precision    recall  f1-score   support

           C     0.7182    0.7886    0.7517    646157
           E     0.8230    0.8700    0.8458    563979
           H     0.8648    0.9089    0.8863    682331
           B     0.7782    0.2634    0.3936     26516
           G     0.6694    0.4889    0.5651     72668
           I     0.6053    0.2415    0.3452       381
           T     0.6482    0.5901    0.6178    255238
           S     0.6418    0.4285    0.5139    210849

    accuracy                         0.7746   2458119
   macro avg     0.7186    0.5725    0.6149   2458119
weighted avg     0.7683    0.7746    0.7669   2458119

Q8 Accuracy: 0.7746


**DenseNet Model**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data_cleaned['seq']
# y = pss_data_cleaned['sst8']

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 3. Encoding Sequences & Labels
# # =========================================================
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128
# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 4. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 5. DenseNet1D Model
# # =========================================================
# class DenseBlock(nn.Module):
#     def __init__(self, in_channels, growth_rate):
#         super(DenseBlock, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels, growth_rate, kernel_size=3, padding=1)
#         self.relu = nn.ReLU()
#     def forward(self, x):
#         out = self.relu(self.conv1(x))
#         out = torch.cat([x, out], dim=1)  # Dense connection
#         return out

# class DenseNet1D(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_labels, growth_rate=32, num_blocks=3, pad_idx=0):
#         super(DenseNet1D, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.blocks = nn.ModuleList()
#         channels = embed_dim
#         for _ in range(num_blocks):
#             self.blocks.append(DenseBlock(channels, growth_rate))
#             channels += growth_rate
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(channels, num_labels)
#     def forward(self, input_ids, labels=None):
#         x = self.embedding(input_ids)        # (batch, seq_len, embed_dim)
#         x = x.transpose(1, 2)                # (batch, embed_dim, seq_len)
#         for block in self.blocks:
#             x = block(x)
#         x = x.transpose(1, 2)                # back to (batch, seq_len, channels)
#         x = self.dropout(x)
#         logits = self.classifier(x)          # (batch, seq_len, num_labels)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 6. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# num_labels = len(label_map)

# model = DenseNet1D(vocab_size, embed_dim, num_labels).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# epochs = 5

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)
#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)
#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

# # =========================================================
# # 7. Evaluation: Q8 Accuracy & Classification Report
# # =========================================================
# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)
#         logits = model(input_ids)
#         preds = torch.argmax(logits, dim=-1)
#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

/tmp/ipykernel_19/3184646567.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 1.3102 | Val Loss: 1.2511
Epoch 2/5 | Train Loss: 1.2718 | Val Loss: 1.2441
Epoch 3/5 | Train Loss: 1.2624 | Val Loss: 1.2305
Epoch 4/5 | Train Loss: 1.2579 | Val Loss: 1.2354
Epoch 5/5 | Train Loss: 1.2551 | Val Loss: 1.2291


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           C     0.5530    0.5615    0.5572    646157
           E     0.5850    0.6032    0.5940    563979
           H     0.5430    0.7950    0.6453    682331
           B     0.9524    0.0030    0.0060     26516
           G     0.7612    0.0596    0.1105     72668
           I     0.0000    0.0000    0.0000       381
           T     0.4240    0.3296    0.3709    255238
           S     0.4908    0.0404    0.0747    210849

    accuracy                         0.5462   2458119
   macro avg     0.5387    0.2990    0.2948   2458119
weighted avg     0.5492    0.5462    0.5101   2458119

Q8 Accuracy: 0.5462


**VGG Model**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# X = pss_data_cleaned['seq']
# y = pss_data_cleaned['sst8']

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 3. Encoding Sequences & Labels
# # =========================================================
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1
# pad_idx = 0

# def encode_sequence(seq, max_len=128):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=128):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# max_len = 128
# train_enc = [encode_sequence(s, max_len) for s in train_texts]
# val_enc   = [encode_sequence(s, max_len) for s in val_texts]
# test_enc  = [encode_sequence(s, max_len) for s in test_texts]

# train_lab = [encode_labels(l, max_len) for l in train_labels]
# val_lab   = [encode_labels(l, max_len) for l in val_labels]
# test_lab  = [encode_labels(l, max_len) for l in test_labels]

# # =========================================================
# # 4. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=32)
# test_loader  = DataLoader(test_dataset, batch_size=32)

# # =========================================================
# # 5. VGG16-1D Model (with padding to maintain seq_len)
# # =========================================================
# class VGG16_1D(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_labels, pad_idx=0):
#         super(VGG16_1D, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         self.features = nn.Sequential(
#             # Block 1
#             nn.Conv1d(embed_dim, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(64, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=2, stride=1, padding=1),
#             # Block 2
#             nn.Conv1d(64, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(128, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=2, stride=1, padding=1),
#             # Block 3
#             nn.Conv1d(128, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=2, stride=1, padding=1),
#             # Block 4
#             nn.Conv1d(256, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=2, stride=1, padding=1),
#             # Block 5
#             nn.Conv1d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.Conv1d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=2, stride=1, padding=1)
#         )

#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(512, num_labels)

#     def forward(self, input_ids, labels=None):
#       x = self.embedding(input_ids)    # (batch, seq_len, embed_dim)
#       x = x.transpose(1, 2)            # (batch, embed_dim, seq_len)
#       x = self.features(x)
#       x = x.transpose(1, 2)            # (batch, seq_len, channels)
#       x = self.dropout(x)
#       logits = self.classifier(x)

#       # Trim logits to max_len if necessary
#       logits = logits[:, :max_len, :]

#       loss = None
#       if labels is not None:
#           loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#           loss = loss_fct(logits.reshape(-1, logits.size(-1)), labels.reshape(-1))
#       return (loss, logits) if loss is not None else logits


# # =========================================================
# # 6. Training Loop
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# num_labels = len(label_map)

# model = VGG16_1D(vocab_size, embed_dim, num_labels).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# epochs = 5

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)
#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)
#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipykernel_19/2323266229.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 1.0649 | Val Loss: 0.9101
Epoch 2/5 | Train Loss: 0.8722 | Val Loss: 0.8405
Epoch 3/5 | Train Loss: 0.8162 | Val Loss: 0.8038
Epoch 4/5 | Train Loss: 0.7874 | Val Loss: 0.7836
Epoch 5/5 | Train Loss: 0.7687 | Val Loss: 0.7678


In [ ]:
# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids, labels=None)  # <-- fixed
#         preds = torch.argmax(logits, dim=-1)

#         for p, l in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p, l):
#                 if li != -100:  # ignore padding
#                     all_preds.append(pi)
#                     all_labels.append(li)

# from sklearn.metrics import classification_report, accuracy_score

# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.6555    0.7564    0.7024    646157
           E     0.7962    0.8086    0.8024    563979
           H     0.8178    0.8997    0.8568    682331
           B     0.8004    0.2380    0.3670     26516
           G     0.6949    0.3766    0.4885     72668
           I     0.5620    0.2021    0.2973       381
           T     0.5864    0.5571    0.5714    255238
           S     0.6658    0.3131    0.4259    210849

    accuracy                         0.7325   2458119
   macro avg     0.6974    0.5190    0.5639   2458119
weighted avg     0.7293    0.7325    0.7209   2458119

Q8 Accuracy: 0.7325


**CNN+BiLSTM Model**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# # =========================================================
# # 2. Train / Val / Test Split
# # =========================================================
# label_map = {'C': 0, 'E': 1, 'H': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}

# # =========================================================
# # 3. Train / Val / Test Split
# # =========================================================
# X = pss_data['seq']   # amino acid sequences
# y = pss_data['sst8']  # secondary structure labels

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# print("Train size:", len(train_texts))
# print("Val size:", len(val_texts))
# print("Test size:", len(test_texts))

# # =========================================================
# # 3. Encoding Sequences & Labels
# # =========================================================
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab) + 1
# pad_idx = 0
# max_len = 128

# def encode_sequence(seq, max_len=max_len):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     ids = ids[:max_len] + [pad_idx] * (max_len - len(ids))
#     return ids

# def encode_labels(labels, max_len=max_len):
#     ids = [label_map.get(l, -100) for l in labels]
#     ids = ids[:max_len] + [-100] * (max_len - len(ids))
#     return ids

# train_enc = [encode_sequence(s) for s in train_texts]
# val_enc   = [encode_sequence(s) for s in val_texts]
# test_enc  = [encode_sequence(s) for s in test_texts]

# train_lab = [encode_labels(l) for l in train_labels]
# val_lab   = [encode_labels(l) for l in val_labels]
# test_lab  = [encode_labels(l) for l in test_labels]

# # =========================================================
# # 4. Torch Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
# val_loader   = DataLoader(val_dataset, batch_size=32, pin_memory=True)
# test_loader  = DataLoader(test_dataset, batch_size=32, pin_memory=True)

# # =========================================================
# # 5. CNN + BiLSTM Model
# # =========================================================
# class CNN_BiLSTM(nn.Module):
#     def __init__(self, vocab_size, embed_dim, cnn_channels, lstm_hidden_dim, num_labels, pad_idx=0):
#         super(CNN_BiLSTM, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

#         # CNN layers
#         self.cnn = nn.Sequential(
#             nn.Conv1d(embed_dim, cnn_channels, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.Conv1d(cnn_channels, cnn_channels, kernel_size=3, padding=1),
#             nn.ReLU()
#         )

#         # BiLSTM
#         self.bilstm = nn.LSTM(
#             input_size=cnn_channels,
#             hidden_size=lstm_hidden_dim,
#             num_layers=1,
#             bidirectional=True,
#             batch_first=True
#         )

#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(lstm_hidden_dim*2, num_labels)

#     def forward(self, input_ids, labels=None):
#         x = self.embedding(input_ids)        # (batch, seq_len, embed_dim)
#         x = x.transpose(1, 2)                # (batch, embed_dim, seq_len) for CNN
#         x = self.cnn(x)
#         x = x.transpose(1, 2)                # (batch, seq_len, cnn_channels)
#         x, _ = self.bilstm(x)                # (batch, seq_len, lstm_hidden*2)
#         x = self.dropout(x)
#         logits = self.classifier(x)
#         logits = logits[:, :max_len, :]      # trim to max_len

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.reshape(-1, logits.size(-1)), labels.reshape(-1))
#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 6. Training
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# vocab_size = len(aa_vocab) + 1
# embed_dim = 128
# cnn_channels = 128
# lstm_hidden_dim = 256
# num_labels = len(label_map)

# model = CNN_BiLSTM(vocab_size, embed_dim, cnn_channels, lstm_hidden_dim, num_labels).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# epochs = 5

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)
#         optimizer.zero_grad()
#         loss, logits = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)
#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

/tmp/ipykernel_19/1856346211.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Train size: 176490
Val size: 22061
Test size: 22062
Epoch 1/5 | Train Loss: 0.9248 | Val Loss: 0.7553
Epoch 2/5 | Train Loss: 0.7176 | Val Loss: 0.6677
Epoch 3/5 | Train Loss: 0.6588 | Val Loss: 0.6359
Epoch 4/5 | Train Loss: 0.6276 | Val Loss: 0.6160
Epoch 5/5 | Train Loss: 0.6085 | Val Loss: 0.6042


In [ ]:
# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)

#         logits = model(input_ids, labels=None)  # <-- fixed
#         preds = torch.argmax(logits, dim=-1)

#         # Flatten predictions and labels, ignore padding
#         for p_seq, l_seq in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p_seq, l_seq):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# from sklearn.metrics import classification_report, accuracy_score

# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

              precision    recall  f1-score   support

           C     0.7458    0.7884    0.7665    646157
           E     0.8298    0.8960    0.8616    563979
           H     0.8690    0.9328    0.8997    682331
           B     0.8344    0.2682    0.4059     26516
           G     0.7206    0.5150    0.6007     72668
           I     0.9231    0.1260    0.2217       381
           T     0.6796    0.6051    0.6402    255238
           S     0.6656    0.4606    0.5445    210849

    accuracy                         0.7922   2458119
   macro avg     0.7835    0.5740    0.6176   2458119
weighted avg     0.7858    0.7922    0.7843   2458119

Q8 Accuracy: 0.7922


**Attention Mechanism+CNN**

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # =========================================================
# # 1. Load & Clean Dataset
# # =========================================================
# pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')
# pss_data_cleaned = pss_data.dropna(subset=['sst8'])

# label_map = {'C':0,'E':1,'H':2,'B':3,'G':4,'I':5,'T':6,'S':7}
# num_labels = len(label_map)
# pad_idx = 0
# max_len = 128

# # =========================================================
# # 2. Train/Val/Test Split
# # =========================================================
# X = pss_data_cleaned['seq']
# y = pss_data_cleaned['sst8']

# train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# # =========================================================
# # 3. Encoding
# # =========================================================
# aa_vocab = {aa: idx+1 for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# aa_vocab["X"] = len(aa_vocab)+1

# def encode_sequence(seq, max_len=max_len):
#     ids = [aa_vocab.get(aa, aa_vocab["X"]) for aa in seq]
#     return ids[:max_len] + [pad_idx]*(max_len - len(ids))

# def encode_labels(labels, max_len=max_len):
#     ids = [label_map.get(l,-100) for l in labels]
#     return ids[:max_len] + [-100]*(max_len - len(ids))

# train_enc = [encode_sequence(s) for s in train_texts]
# val_enc   = [encode_sequence(s) for s in val_texts]
# test_enc  = [encode_sequence(s) for s in test_texts]

# train_lab = [encode_labels(l) for l in train_labels]
# val_lab   = [encode_labels(l) for l in val_labels]
# test_lab  = [encode_labels(l) for l in test_labels]

# # =========================================================
# # 4. Dataset
# # =========================================================
# class PSSDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.encodings)
#     def __getitem__(self, idx):
#         return {
#             "input_ids": torch.tensor(self.encodings[idx], dtype=torch.long),
#             "labels": torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# train_dataset = PSSDataset(train_enc, train_lab)
# val_dataset   = PSSDataset(val_enc, val_lab)
# test_dataset  = PSSDataset(test_enc, test_lab)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
# val_loader   = DataLoader(val_dataset, batch_size=32, pin_memory=True)
# test_loader  = DataLoader(test_dataset, batch_size=32, pin_memory=True)

# # =========================================================
# # 5. CNN + Attention Model
# # =========================================================
# class CNNAttentionModel(nn.Module):
#     def __init__(self, vocab_size, embed_dim, cnn_channels, num_labels, num_heads=4, pad_idx=0):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
#         self.cnn = nn.Conv1d(embed_dim, cnn_channels, kernel_size=3, padding=1)
#         self.relu = nn.ReLU()
#         self.attention = nn.MultiheadAttention(embed_dim=cnn_channels, num_heads=num_heads, batch_first=True)
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(cnn_channels, num_labels)

#     def forward(self, input_ids, labels=None):
#         x = self.embedding(input_ids)          # (batch, seq_len, embed_dim)
#         x = x.transpose(1,2)                   # (batch, embed_dim, seq_len)
#         x = self.relu(self.cnn(x))
#         x = x.transpose(1,2)                   # (batch, seq_len, cnn_channels)

#         attn_out, _ = self.attention(x, x, x)  # self-attention
#         x = self.dropout(attn_out)
#         logits = self.classifier(x)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
#             loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
#         return (loss, logits) if loss is not None else logits

# # =========================================================
# # 6. Training
# # =========================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# vocab_size = len(aa_vocab)+1
# embed_dim = 128
# cnn_channels = 128
# num_heads = 4

# model = CNNAttentionModel(vocab_size, embed_dim, cnn_channels, num_labels, num_heads).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# epochs = 5

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)
#         optimizer.zero_grad()
#         loss, _ = model(input_ids, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     avg_train_loss = total_loss / len(train_loader)

#     # Validation
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             labels = batch["labels"].to(device)
#             loss, _ = model(input_ids, labels)
#             val_loss += loss.item()
#     avg_val_loss = val_loss / len(val_loader)
#     print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

# # =========================================================
# # 7. Evaluation
# # =========================================================
# model.eval()
# all_preds, all_labels = [], []

# with torch.no_grad():
#     for batch in test_loader:
#         input_ids = batch["input_ids"].to(device)
#         labels = batch["labels"].to(device)
#         logits = model(input_ids, labels=None)
#         preds = torch.argmax(logits, dim=-1)

#         for p_seq, l_seq in zip(preds.cpu().numpy(), labels.cpu().numpy()):
#             for pi, li in zip(p_seq, l_seq):
#                 if li != -100:
#                     all_preds.append(pi)
#                     all_labels.append(li)

# target_names = list(label_map.keys())
# print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
# q8_accuracy = accuracy_score(all_labels, all_preds)
# print(f"Q8 Accuracy: {q8_accuracy:.4f}")

/tmp/ipykernel_19/44030190.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pss_data = pd.read_csv('/kaggle/input/maindataaaaaaaaa/2022-08-03-ss.processed.csv')


Epoch 1/5 | Train Loss: 1.3023 | Val Loss: 1.2949
Epoch 2/5 | Train Loss: 1.2337 | Val Loss: 1.2188
Epoch 3/5 | Train Loss: 1.2134 | Val Loss: 1.2071
Epoch 4/5 | Train Loss: 1.2036 | Val Loss: 1.2087
Epoch 5/5 | Train Loss: 1.1968 | Val Loss: 1.1959
              precision    recall  f1-score   support

           C     0.5250    0.5825    0.5523    646157
           E     0.6535    0.6018    0.6266    563979
           H     0.5484    0.8276    0.6596    682331
           B     0.7758    0.0946    0.1687     26516
           G     0.6405    0.1276    0.2128     72668
           I     0.0000    0.0000    0.0000       381
           T     0.5005    0.2740    0.3541    255238
           S     0.5949    0.0979    0.1682    210849

    accuracy                         0.5626   2458119
   macro avg     0.5298    0.3258    0.3428   2458119
weighted avg     0.5705    0.5626    0.5313   2458119

Q8 Accuracy: 0.5626
